# Praca domowa 6
## Hotel booking 

Karolina Seweryn


Zbudowano trzy typy modeli opartych o drzewa decyzyjne przewidujace ceny doby hotelowej w hotelach w Portugali. Modele:

* Random Forest,
* XGBoost,
* LightGBM.

Poniższe wykresy prezentują wykresy `Partial Dependence Profiles` i `Accumulated Local Dependence` obliczone z użyciem pakietu `DALEX`.

1. RandomForest

![](plots/plot1.png)

2. XGBoost
![](plots/plot2.png)

3. LightGBM 
![](plots/plot3.png)

### Analiza wyników

W przypadku tych modeli wykresy dla zmiennej `arrival_date_month` oznaczającej miesiąc przyjazdu są bardzo intuicyjne i podobne do siebie. Ceny nocy hotelowej są większe w sezonie turystycznym (miesiące letnie). Natomiast w miesiącach zimowych cena spada. Warto zauważyć, że według tych trzech modeli średnio największa cena będzie w sierpniu. Różnice w zachowaniach modeli możemy dostrzec w miesiącach odpowiadającym przełomowi jesieni i zimy. Wtedy model `Random Forest` średnio przewiduje ceny podobne do tych w styczniu czy lutym, zaś modele `XGBoost` i `LightGBM` wyższe. 

Drugą analizowaną zmienną jest `lead_time`, czyli liczba dni pomiędzy datą rezerwacji i przyjazdu. Według pierwszego modelu im większe wartości tej zmiennej tym mniejsza średnia cena za noc w hotelu. Można z tego wywnioskować, że warto wcześnie rezerwować hotele. Podobnie zachowuje się model `LightGBM`. Różnice można dostrzec na początku wykresu, ponieważ w modelu `LightGBM` wychodzi, że średnio reserwacja na ostatnią chwilę może dać nam korzystniejszą cenę niż zakup miesiąc wcześniej. Trochę inny trend można zaobserwować w modelu `XGBoost`. Cena wraz ze wzrostem zmiennej `lead_time` maleje aż do pewnego momentu. W punkcie około 350 dni (około rok przed przyjazdem) średnia cena rośnie. Możliwe, że hotele chcą wykorzystać okazję, wiedząc, że klienci wrócą do hotelu na kolejne wakacje podnoszą cenę za noc w celu zwiększenia zysków.

### Załącznik

Poniżej znajdują się kody z rozwiązaniem pracy domowej.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
from lime.lime_tabular import LimeTabularExplainer

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from ceteris_paribus.explainer import explain
from ceteris_paribus.plots.plots import plot_notebook, plot
from ceteris_paribus.profiles import individual_variable_profile
from dalex.explainer import Explainer

def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values.
    
    le: a fitted sklearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res


def explain_model(model, X_train, X_test, y_train, y_test, obs, selected_variables, columns):
    explainer = explain(model, columns, X_train, y_train)
    cp = individual_variable_profile(explainer, X_train.loc[obs], y_train.loc[obs])
    plot_notebook(cp, selected_variables=selected_variables, print_observations=False)
    
def plot_feature_importance(model, X, y, n_repeats=10, n_jobs=1, title="Permutation Importances"):
    result = permutation_importance(model, X, y, n_repeats=n_repeats,
                                random_state=42, n_jobs=n_jobs)
    sorted_idx = result.importances_mean.argsort()

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.boxplot(result.importances[sorted_idx].T,
               vert=False, labels=X.columns[sorted_idx])
    ax.set_title(title)
    fig.tight_layout()
    plt.show()

def plot_ald_pdp(model, X_train, X_test, y_train, y_test, selected_variables, title):
    explainer = Explainer(model=model, data=X_train, y=y_train, model_type='regression')
    pdp = explainer.model_profile(type = 'partial', variables=selected_variables)
    pdp.plot(title=str("PDP"+title))
    plt.show()
    ale = explainer.model_profile(type = 'accumulated', variables=selected_variables)
    ale.plot(title=str("ALE"+title))
    

In [2]:
df = pd.read_csv("./data/hotel_bookings.csv")

month_name_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
df['arrival_date_month'] = [month_name_to_num[x] for x in df['arrival_date_month']]
df["arrival_weekday"] = [calendar.weekday(df.loc[i, 'arrival_date_year'], df.loc[i, 'arrival_date_month'], df.loc[i, 'arrival_date_day_of_month']) for i in df.index]

df = df.drop(["arrival_date_year", "country", "agent", "company", "reservation_status_date", "reservation_status", "is_canceled"], axis=1)
feature_type = df.dtypes
object_features = [i for i in feature_type.index if feature_type[i] == 'object']

for feat in object_features:
    le = LabelEncoder()
    df[feat] = le.fit_transform(df[feat]) 
    integerMapping = get_integer_mapping(le)
    print(feat)
    print(integerMapping)
    
X = df.drop('adr', axis=1)
X = X.fillna(value=0)
y = df['adr']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)


hotel
{'City Hotel': 0, 'Resort Hotel': 1}
meal
{'BB': 0, 'FB': 1, 'HB': 2, 'SC': 3, 'Undefined': 4}
market_segment
{'Aviation': 0, 'Complementary': 1, 'Corporate': 2, 'Direct': 3, 'Groups': 4, 'Offline TA/TO': 5, 'Online TA': 6, 'Undefined': 7}
distribution_channel
{'Corporate': 0, 'Direct': 1, 'GDS': 2, 'TA/TO': 3, 'Undefined': 4}
reserved_room_type
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'L': 8, 'P': 9}
assigned_room_type
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'P': 11}
deposit_type
{'No Deposit': 0, 'Non Refund': 1, 'Refundable': 2}
customer_type
{'Contract': 0, 'Group': 1, 'Transient': 2, 'Transient-Party': 3}


In [3]:
# Model 1
rf_sel = RandomForestRegressor(max_depth=5, n_estimators=100)
selector = RFE(rf_sel, 15, step=2)
selector.fit(X_train, y_train)
cols = selector.get_support(indices=True)

X_train, X_test = X_train.iloc[:,cols], X_test.iloc[:,cols]

rf = RandomForestRegressor(max_depth=8, n_estimators=500)
rf.fit(X_train, y_train)

y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf = rf.predict(X_test)

# Model 2
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

y_pred_train_xgb = xgb.predict(X_train)
y_pred_test_xgb = xgb.predict(X_test)


In [4]:
# Model 3
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)

y_pred_train_xgb = lgbm.predict(X_train)
y_pred_test_xgb = lgbm.predict(X_test)

# Model 4
lgbm2 = LGBMRegressor(boosting_type='goss')
lgbm2.fit(X_train, y_train)

y_pred_train_xgb = lgbm2.predict(X_train)
y_pred_test_xgb = lgbm2.predict(X_test)

In [5]:
X_train.columns

Index(['hotel', 'lead_time', 'arrival_date_month', 'arrival_date_week_number',
       'stays_in_week_nights', 'adults', 'children', 'meal', 'market_segment',
       'distribution_channel', 'previous_cancellations', 'reserved_room_type',
       'assigned_room_type', 'customer_type', 'total_of_special_requests'],
      dtype='object')